In [1]:
import os
import sys
from dotenv import load_dotenv

# Add the parent directory to sys.path to import the geoencoding module
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from external.geoencoding import GoogleMapsGeocoder

# Initialize the geocoder
geocoder = GoogleMapsGeocoder()

# Test address
test_address = "453 Broome St, NYC"

try:
    # Get the geocoded result
    result_geo = geocoder.geocode_address(test_address)
    
    # Pretty print the result
    print("Geocoding Result:")
    print("-" * 50)
    for key, value in result_geo.items():
        # Format floating point numbers to 6 decimal places if they are coordinates
        if isinstance(value, float):
            print(f"{key}: {value:.6f}")
        else:
            print(f"{key}: {value}")
            
except Exception as e:
    print(f"Error: {str(e)}")


Geocoding Result:
--------------------------------------------------
address: 453 Broome St, NYC
latitude: 40.722066
longitude: -74.000931
formatted_address: 453 Broome Street, New York, NY, 10013, USA
country: USA
county: New York
state: NY


In [2]:
from external.risk_lookup import RiskLookup

risk_lookup = RiskLookup()

test_location = risk_lookup.get_location_risks(
    {
        'country': 'USA',
        'county': 'Ada',
        'state': 'ID'
    }
)

test_location





{'Flooding': 'Relatively High',
 'Cold Wave / Severe Winter Weather': 'Relatively High',
 'Hail': 'Relatively Low',
 'Wildfire': 'Relatively High',
 'Wind': 'Relatively Moderate',
 'Earthquake': 'Relatively Moderate'}

In [3]:
from typing import Dict, Optional
from external.question_master import QuestionMaster
from external.grader import RiskGrader

# Initialize the classes
question_master = QuestionMaster()
grader = RiskGrader()

# Test QuestionMaster with Ada County, ID risks
test_risks: Dict[str, Optional[str]] = {
    'Flooding': 'Relatively High',
    'Cold Wave / Severe Winter Weather': 'Relatively High',
    'Hail': 'Relatively Low',
    'Wildfire': 'Relatively Low',
    'Wind': 'Relatively Moderate',
    'Earthquake': 'Relatively Moderate'
}

print("Testing QuestionMaster.get_relevant_questions():")
print("-" * 50)

# Get questions for high-risk categories
questions = question_master.get_relevant_questions(test_risks)

print(f"Found {len(questions)} questions for high-risk categories:")
high_risk_categories = set()
for i, q in enumerate(questions, 1):
    print(f"\n{i}. {q['question']}")
    print(f"   Risk Type: {q['risk_type']}")
    print(f"   Importance: {q['importance']}")
    high_risk_categories.add(q['risk_type'])

print(f"\nHigh risk categories covered: {', '.join(sorted(high_risk_categories))}")

# Verify that we only got questions for high-risk categories
expected_high_risk = {'Flooding', 'Winter'}  # From test_risks
assert high_risk_categories == expected_high_risk, f"Expected {expected_high_risk}, got {high_risk_categories}"


Testing QuestionMaster.get_relevant_questions():
--------------------------------------------------
Found 20 questions for high-risk categories:

1. Are all major systems (e.g. HVAC, water heater, furnace) elevated at least 1 foot above the ground?
   Risk Type: Flooding
   Importance: High

2. Do you have a sump pump installed in your home?
   Risk Type: Flooding
   Importance: High

3. Is your basement properly sealed and waterproofed? Including no cracks and leaks.
   Risk Type: Flooding
   Importance: High

4. Does your home have flood vents or openings to allow water to drain from the foundation of your home?
   Risk Type: Flooding
   Importance: Medium

5. Is the foundation of your home elevated at least 1 foot above the ground?
   Risk Type: Flooding
   Importance: High

6. Is your first floor/basement walls and flooring made with flood-resistant materials?
   Risk Type: Flooding
   Importance: Medium

7. Have all windows and doors been sealed with weather stripping and/or caulk

In [4]:
print("\nTesting RiskGrader.calculate_score():")
print("-" * 50)

# Create test answers that include risk levels
test_answers = [
    # Very High Risk Questions (10 points base)
    {
        'question': 'Do you have a sump pump installed in your home?',
        'risk_type': 'Flooding',
        'importance': 'High',
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Very High'
    },
    {
        'question': 'Is your basement properly sealed and waterproofed?',
        'risk_type': 'Flooding',
        'importance': 'Medium',  # 70% multiplier
        'answer': 'No',  # Incorrect answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Very High'
    },
    # Relatively High Risk Questions (7 points base)
    {
        'question': 'Are your gutters clear of debris?',
        'risk_type': 'Winter',
        'importance': 'High',
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Relatively High'
    },
    {
        'question': 'Do you have fire resistant materials within 5 feet of your home?',
        'risk_type': 'Wildfire',
        'importance': 'Low',  # 40% multiplier
        'answer': 'Yes',  # Correct answer
        'rubric': {'Yes': 1, 'No': 0},
        'risk_level': 'Relatively High'
    }
]

# Calculate scores
results = grader.calculate_score(test_answers)

# Print results
print(f"\nFinal Score: {results['total_score']}%")
print(f"Points Earned: {results['points_earned']} out of {results['points_possible']}")
print("\nBreakdown by Risk Level:")
for level, stats in results['breakdown'].items():
    print(f"\n{level}:")
    print(f"  Earned: {stats['earned']} points")
    print(f"  Possible: {stats['possible']} points")
    print(f"  Score: {stats['percentage']}%")

# Verify calculations
expected_points = {
    'Very High': {
        'earned': 10,  # First question correct (10 * 1.0)
        'possible': 17  # First question (10 * 1.0) + Second question (10 * 0.7)
    },
    'Relatively High': {
        'earned': 9.8,  # First question (7 * 1.0) + Second question (7 * 0.4)
        'possible': 9.8  # Same as earned since both answered correctly
    }
}

for level, expected in expected_points.items():
    actual = results['breakdown'][level]
    assert abs(actual['earned'] - expected['earned']) < 0.01, \
        f"{level} earned points: expected {expected['earned']}, got {actual['earned']}"
    assert abs(actual['possible'] - expected['possible']) < 0.01, \
        f"{level} possible points: expected {expected['possible']}, got {actual['possible']}"

print("\nAll assertions passed! Calculations are correct.")



Testing RiskGrader.calculate_score():
--------------------------------------------------

Final Score: 73.88%
Points Earned: 19.8 out of 26.8

Breakdown by Risk Level:

Very High:
  Earned: 10.0 points
  Possible: 17.0 points
  Score: 58.82%

Relatively High:
  Earned: 9.8 points
  Possible: 9.8 points
  Score: 100.0%

All assertions passed! Calculations are correct.


In [6]:
import os
import sys
from pathlib import Path

# Add the parent directory to sys.path to import the camera module
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from external.camera import RiskPhotoManager, RiskPhotoValidator

# Initialize the classes
photo_manager = RiskPhotoManager()
validator = RiskPhotoValidator()

# Test configuration
question = "Are there any tree limbs or branches hanging over your home?"
question_id = "tree_limbs_over_home"
user_id = "test_user_123"
user_answer = "Yes"  # Based on the image, there is clearly a tree over the home
rubric = {"Yes": 0, "No": 1}  # From risk_questions.json - "Yes" indicates a risk

print("Testing RiskPhotoManager and RiskPhotoValidator:")
print("-" * 50)

# Get the absolute path to example_photo.jpg
notebook_dir = Path(os.getcwd())
photo_path = notebook_dir / "example_photo.jpg"

try:
    # Upload the photo
    with open(photo_path, "rb") as f:
        photo_data = f.read()
        photo_url = photo_manager.upload_photo(photo_data, question_id, user_id)
        
    if photo_url:
        print(f"\n✅ Successfully uploaded photo: {photo_url}")
        
        # Get all photos for this question
        photos = photo_manager.get_photos_for_question(question_id, user_id)
        print(f"\nFound {len(photos)} photos for question {question_id}")
        
        # First validate the photo quality
        validation = validator.validate_photos(photos, question)
        print("\nPhoto Validation Results:")
        print(f"Valid: {validation.get('is_valid', False)}")
        print(f"Analysis: {validation.get('analysis', 'N/A')}")
        print(f"Concerns: {', '.join(validation.get('concerns', []))}")
        
        # Then verify the answer
        verification = validator.verify_answer(photos, question, user_answer, rubric)
        print("\nAnswer Verification Results:")
        print(f"Verified: {verification.get('verified', False)}")
        print(f"Matches Answer: {verification.get('matches_answer', False)}")
        print(f"Confidence: {verification.get('confidence', 0.0)}")
        print(f"Analysis: {verification.get('analysis', 'N/A')}")
        print(f"Correct Answer: {verification.get('correct_answer', 'N/A')}")
        print(f"Evidence: {verification.get('evidence', 'N/A')}")
        print(f"Concerns: {', '.join(verification.get('concerns', []))}")
        
        # Clean up - delete the test photo
        if photo_manager.delete_photo(photo_url):
            print("\n✅ Successfully deleted test photo")
        else:
            print("\n❌ Failed to delete test photo")
            
except Exception as e:
    print(f"\n❌ Error during testing: {str(e)}")


Testing RiskPhotoManager and RiskPhotoValidator:
--------------------------------------------------
Error uploading photo: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist
